![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module ?: IPXact Example
#### Written by Paul Rigge ([rigge@berkeley.edu](mailto:rigge@berkeley.edu))

Some explanation should go here

In [ ]:
//unmanaged dep for ipxact
interp.load.cp(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(System.getProperty("user.dir") + "/../rocket-dsp-utils-assembly-1.0.jar")))


In [ ]:
import chisel3._
import chisel3.util._

class FIRIO(val w: Int) extends Bundle {
    val in  = Flipped(Decoupled(UInt(w.W)))
    val out = Decoupled(UInt(w.W))
}

class FIR(val w: Int, queueDepth: Int = 4) extends Module {
    val io = IO(new FIRIO(w))
    
    io.out <> Queue(io.in, queueDepth)
}
println(chisel3.Driver.emit(() => new FIR(64)))

## DspBlock


In [ ]:
import dspblocks._
import freechips.rocketchip.amba.axi4._
import freechips.rocketchip.amba.axi4stream._
import freechips.rocketchip.config._
import freechips.rocketchip.diplomacy._

case object FIRQueueDepth extends CSRField {
    val name = "firQueueDepth"
}

case class FIRTap(idx: Int) extends CSRField {
    val name = s"firTap$idx"
}

class FIRBlock(queueDepth: Int, nTaps: Int)(implicit p: Parameters) extends
AXI4DspBlock with AXI4HasCSR with HasIPXactParameters {
    outer =>
    addStatus(FIRQueueDepth)
    
    for (i <- 0 until nTaps) {
        addControl(FIRTap(i))
    }
    makeCSRs()
    
    val streamNode = AXI4StreamIdentityNode()
    
    override def ipxactParameters = Map(
        "queueDepth" -> queueDepth.toString,
        "nTaps"      -> nTaps.toString
    )

    def beatBytes: Int = 8
    def csrAddress = freechips.rocketchip.diplomacy.AddressSet(0x0, 0xffffL)
    def csrBase: Int = 0
    def csrSize: Int = 4096
    
    lazy val module = new LazyModuleImp(this) {
        val (in, _)  = streamNode.in(0)
        val (out, _) = streamNode.out(0)
        val mem = outer.mem.map { m => m.in(0) }
        
        val width = in.params.n * 8
        status(FIRQueueDepth) := queueDepth.U
        
        val fir = Module(new FIR(width, queueDepth))
        
        fir.io.in.valid  := in.valid
        fir.io.in.bits   := in.bits.data
        in.ready         := fir.io.in.ready
        

        out.valid        := fir.io.out.valid
        out.bits.data    := fir.io.out.bits
        fir.io.out.ready := out.ready
    }
}

In [ ]:
object FIRGenerator extends ipxact.IPXactGeneratorApp with App {
  override val verilogFilename: String = "BlindModule.v"
  override val ipxactDir: String = "./"
  implicit val p: Parameters = Parameters.root((new freechips.rocketchip.coreplex.BaseCoreplexConfig).toInstance)

    val blindNodes = DspBlockBlindNodes.apply(
      AXI4StreamBundleParameters(
        n = 8,
        i = 1,
        d = 1,
        u = 1,
        hasData = true,
        hasStrb = true,
        hasKeep = true
      ),
      () => AXI4MasterNode(Seq(AXI4MasterPortParameters(Seq(AXI4MasterParameters("fir"))))))
    
    val dut = () => {
        val lazyMod = LazyModule(DspBlock.blindWrapper(() => new FIRBlock(4, 8), blindNodes))
        val m = lazyMod.module
        IPXactComponents._ipxactComponents += DspIPXact.makeDspBlockComponent(lazyMod.internal)
        m
    }
    
    chisel3.Driver.dumpFirrtl(chisel3.Driver.elaborate(dut), None)
    generateIPXact(IPXactComponents.ipxactComponents())
}
FIRGenerator.main(Array[String]())

## IPXact output
Look in the file browsers for a file called `craft_BlindModule_edu.berkeley.cs_1.0.xml`.